# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [27]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
import plotly.graph_objects as go

cf.go_offline()

In [28]:
data = pd.read_excel('../data/Online Retail.xlsx')

In [29]:
data

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.20,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.60,13408,United Kingdom
...,...,...,...,...,...,...,...,...,...
396029,580691,2011-12-05 15:48:00,90214W,"LETTER ""W"" BLING KEY RING",12,0.29,3.48,13790,United Kingdom
396030,580691,2011-12-05 15:48:00,90214Z,"LETTER ""Z"" BLING KEY RING",12,0.29,3.48,13790,United Kingdom
396031,580865,2011-12-06 11:58:00,90089,PINK CRYSTAL SKULL PHONE CHARM,12,0.19,2.28,17914,United Kingdom
396032,580865,2011-12-06 11:58:00,90089,PINK CRYSTAL SKULL PHONE CHARM,12,0.19,2.28,17914,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [4]:

data_filt1 = data[(data['Country'] != 'United Kingdom') &
     (data['InvoiceDate'] > '2011-04-01') &
     (data['InvoiceDate'] < '2011-05-01')]

agrupado1 = data_filt1[['Country','Quantity','Revenue']].groupby(['Country']).sum()

agrupado1.iplot(kind='bar',title='quant & revenue by country')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [5]:
data_filt2 = data[(data['Country'] == 'France') &
     (data['InvoiceDate'] > '2011-01-01') &
     (data['InvoiceDate'] < '2011-06-01')]

agrupado2 = data_filt2[['InvoiceDate','Quantity','Revenue']].groupby(['InvoiceDate']).sum()
agrupado2.iplot(kind='line')

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [8]:
data_filt3 = data[(data['Description']=='PARTY BUNTING')]

agrupado3 = data_filt3[['Quantity','UnitPrice','Country']].groupby('Country', as_index=False).mean()


In [9]:
agrupado3.iplot(x='Quantity', y='UnitPrice', categories='Country')

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [11]:

data4 = data[(data['Country'] == 'EIRE') |
     (data['Country'] == 'Germany') |
     (data['Country'] == 'France') |
     (data['Country'] == 'Netherlands')]

data4 = data4[['Country','InvoiceNo','Quantity']]#.groupby(['Country','InvoiceNo']).sum()

data4['EIRE'] = data4[data4['Country']=='EIRE']['Quantity']
data4['Germany'] = data4[data4['Country']=='Germany']['Quantity']
data4['France'] = data4[data4['Country']=='France']['Quantity']
data4['Netherlands'] = data4[data4['Country']=='Netherlands']['Quantity']
data4 = data4.drop(columns=['InvoiceNo','Country'])


data4 = data.pivot_table(values='Quantity', columns='Country', 
                      index='customerID', aggfunc='sum')

data4.iplot(kind='histogram',bins=30)

KeyError: 'customerID'

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [12]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']
country_list = ['EIRE', 'Germany', 'France', 'Netherlands']


data5 = data[((data['Country'] == 'EIRE') |
     (data['Country'] == 'Germany') |
     (data['Country'] == 'France') |
     (data['Country'] == 'Netherlands')) &
        ((data['Description'] == 'JUMBO BAG RED RETROSPOT') |
        (data['Description'] == 'CREAM HANGING HEART T-LIGHT HOLDER') |
        (data['Description'] == 'REGENCY CAKESTAND 3 TIER'))]

data5p = data5.pivot_table(values='Revenue', columns='Country', 
                         index='Description', aggfunc='sum').reset_index()


In [13]:
data5p.iplot(kind='bar', x='Description', xTitle='Product', 
           yTitle='Revenue', title='Revenue by country and product')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [17]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']
#uk = uk['Year','Month']

In [18]:
@interact(Month=list(uk['Month'].unique()), 
          Year=list(uk['Year'].unique())
         )

def liner(Month, Year):
    data6 = uk[(uk['Month']==Month) & 
              (uk['Year']==Year)]
    data6.drop(columns=['InvoiceNo','StockCode','Description','UnitPrice','Revenue','CustomerID','Country','Year','Month','Day'], inplace=True)
    data6.sort_values('InvoiceDate', inplace=True)
    data6.iplot(kind='line', x='InvoiceDate', xTitle='Time', color='Red',
           yTitle='Quantity', title='Quantity in UK by time')

interactive(children=(Dropdown(description='Month', options=(12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=12)…

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [23]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func).reset_index()

In [24]:
products

,Description,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
0,4 PURPLE FLOCK DINNER CANDLES,35,134,2.318421,255.46,30
1,50'S CHRISTMAS GIFT BAG LARGE,100,1721,1.247900,2067.25,98
2,DOLLY GIRL BEAKER,100,661,1.250000,826.25,77
3,I LOVE LONDON MINI BACKPACK,55,181,4.150000,751.15,46
4,NINE DRAWER OFFICE TIDY,25,44,14.761538,628.40,24
...,...,...,...,...,...,...
3615,ZINC T-LIGHT HOLDER STARS SMALL,220,4258,0.839005,3399.62,168
3616,ZINC TOP 2 DOOR WOODEN SHELF,9,10,16.950000,169.50,9
3617,ZINC WILLIE WINKIE CANDLE STICK,169,2007,0.877052,1716.87,123
3618,ZINC WIRE KITCHEN ORGANISER,11,24,6.881818,146.40,11


In [26]:
'''@interact(bins1 = (8, len(products['InvoiceNo'].unique())),
          bins2 = (8, len(products['CustomerID'].unique())),
         )

def scatter(bins1, bins2):
    products.iplot(x='InvoiceNo', y='CustomerID', categories='Description',
           xTitle='Tenure', yTitle='Total Charges',
           title='Charges vs. Tenure: One Year Contract, Credit Card Customers')'''

"@interact(bins1 = (8, len(products['InvoiceNo'].unique())),\n          bins2 = (8, len(products['CustomerID'].unique())),\n         )\n\ndef scatter(bins1, bins2):\n    products.iplot(x='InvoiceNo', y='CustomerID', categories='Description',\n           xTitle='Tenure', yTitle='Total Charges',\n           title='Charges vs. Tenure: One Year Contract, Credit Card Customers')"

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [36]:
@interact(descripcion='WHITE')

def chart(descripcion):
    data8 = data[data['Description'].str.contains(descripcion)]
    data8 = data8[['Description','Revenue']].groupby('Description').agg({'Revenue':'sum'}).reset_index()
    
    data8.iplot(kind='bar', x='Description', xTitle='Revenues', color='green')

interactive(children=(Text(value='WHITE', description='descripcion'), Output()), _dom_classes=('widget-interac…